In [ ]:
import pandas as pd

# Set display option for debugging purposes
pd.set_option("display.max_columns", None)

# Read the excel file
df = pd.read_excel("data/feuille11.xlsx")

# Create a regex pattern to match the codes
code_pattern = '|'.join(["E11.2", "E11.3", "E11.4", "E11.5", "L97"])

# Filter rows where either column contains any of the codes
df_filtered = df[
    df["Code Dx Principal"].str.contains(code_pattern) |
    df["Ecle Me Cim  Diagnostic (Autres)"].str.contains(code_pattern)
]

# Function to replace non-matching codes with NaN
def replace_codes(series, pattern):
    return series.where(series.str.contains(pattern), other=pd.NA)

# Replace the codes in both columns
df_filtered["Code Dx Principal"] = replace_codes(df_filtered["Code Dx Principal"], code_pattern)
df_filtered["Ecle Me Cim  Diagnostic (Autres)"] = replace_codes(df_filtered["Ecle Me Cim  Diagnostic (Autres)"], code_pattern)

# Convert 'Dt Admission' to datetime
df_filtered['Dt Admission'] = pd.to_datetime(df_filtered['Dt  Admission'])

# Sort the dataframe by patient ID and admission date
df_sorted = df_filtered.sort_values(['PatienId', 'Dt  Admission'])

# Add a visit number for each patient ID and admission date combination
df_sorted['Visit_Number'] = df_sorted.groupby(['PatienId', 'Dt  Admission']).cumcount() + 1

# Pivot the dataframe with the additional Visit_Number to ensure uniqueness
df_pivot_principal = df_sorted.pivot(index='PatienId', columns=['Dt  Admission', 'Visit_Number'], values='Code Dx Principal')
df_pivot_secondary = df_sorted.pivot(index='PatienId', columns=['Dt  Admission', 'Visit_Number'], values='Ecle Me Cim  Diagnostic (Autres)')

# Concatenate the pivoted dataframes and align them with the patient IDs
df_pivot = pd.concat([df_pivot_principal, df_pivot_secondary], axis=1, keys=['Principal', 'Secondary'])

# Flatten the MultiIndex columns by joining the level values into one string
new_columns = [
    f'{level0}_{level1.date()}_{level2}' 
    for (level0, level1, level2) in zip(
        df_pivot.columns.get_level_values(0), 
        df_pivot.columns.get_level_values(1), 
        df_pivot.columns.get_level_values(2)
    )
]

df_pivot.columns = new_columns

# Define a function to extract date and visit number from the column name
def extract_date_visit(col_name):
    parts = col_name.split('_')
    # Extract the date and visit number
    date_part = parts[1]  # '2023-01-01'
    visit_part = parts[2]  # 'visit1'
    # Convert to datetime and int for sorting
    date = pd.to_datetime(date_part)
    visit_number = int(visit_part)#.replace('visit', ''))
    return date, visit_number

# Sort the columns using the custom key function
sorted_columns = sorted(df_pivot.columns, key=extract_date_visit)

# Reindex the DataFrame with the sorted columns
df_pivot_sorted = df_pivot[sorted_columns]

# Create column lists

t3 = []
t4 = []
for col_name in df_pivot_sorted.columns:
    
    parts = col_name.split('_')
    # Extract the date and visit number
    
    t3.append('Principal_'+parts[1]+'_'+parts[2])  # '2023-01-01'
    t4.append('Secondary_'+parts[1]+'_'+parts[2])

# Filter only relevant columns to reduce memory usage
relevant_columns = t3 + t4
df_optimized = df_pivot_sorted[relevant_columns]


# Convert all the relevant columns to string type to avoid type mismatch issues
df_optimized[t3 + t4] = df_optimized[t3 + t4].astype(str)


l = ['E11.2','E11.3','E11.4','E11.5','L97']


def process_row(row):
    v0 = []
    for j in t3:
        code_principal = row[j]
        code_secondary = row[j.replace('Principal', 'Secondary')]
        
        # Check for the conditions as per the original script
        if code_principal in l and code_secondary in l:
            if code_principal != code_secondary and code_principal not in v0 and code_secondary not in v0:
                v0.extend([code_principal, code_secondary])
            elif code_principal == code_secondary and code_principal not in v0:
                v0.append(code_principal)
        elif code_principal in l and code_principal not in v0:
            v0.append(code_principal)
        elif code_secondary in l and code_secondary not in v0:
            v0.append(code_secondary)

    return v0
def process_row1(row):
    v0 = []
    for j in t3:
        code_principal = row[j]
        code_secondary = row[j.replace('Principal', 'Secondary')]
        
        # Check for the conditions as per the original script
        if code_principal in l and code_secondary in l:
            if code_principal != code_secondary and code_principal not in v0 and code_secondary not in v0:
                v0.extend([code_secondary,code_principal])
            elif code_principal == code_secondary and code_principal not in v0:
                v0.append(code_principal)
        elif code_principal in l and code_principal not in v0:
            v0.append(code_principal)
        elif code_secondary in l and code_secondary not in v0:
            v0.append(code_secondary)

    return v0
def process_rowa(row):
    v0  = []
    for j in t3:
        code_principal = row[j]
        code_secondary = row[j.replace('Principal', 'Secondary')]
        
        # Check for the conditions as per the original script
        if code_principal in l and code_secondary in l:
            if code_principal != code_secondary and code_principal not in v0 and code_secondary not in v0:
                v0.extend([code_principal])
            elif code_principal == code_secondary and code_principal not in v0:
                v0.append(code_principal)
        elif code_principal in l and code_principal not in v0:
            v0.append(code_principal)
        elif code_secondary in l and code_secondary not in v0:
            v0.append(code_secondary)
    return v0

def process_rowb(row):
    v0b = []
    for j in t3:
        code_principal = row[j]
        code_secondary = row[j.replace('Principal', 'Secondary')]
        
        # Check for the conditions as per the original script
        if code_principal in l and code_secondary in l:
            if code_principal != code_secondary and code_principal not in v0 and code_secondary not in v0:
                v0b.extend([code_secondary])
            elif code_principal == code_secondary and code_principal not in v0b:
                v0b.append(code_principal)
        elif code_principal in l and code_principal not in v0b:
            v0b.append(code_principal)
        elif code_secondary in l and code_secondary not in v0b:
            v0b.append(code_secondary)
    return v0b

# Apply the function to each row

processed_data = df_optimized.apply(process_row, axis=1)
processed_data1 = df_optimized.apply(process_row1, axis=1)
processed_dataa  = df_optimized.apply(process_rowa, axis=1)
processed_datab = df_optimized.apply(process_rowb, axis=1)

# Create a new DataFrame to store the results
la4 = pd.DataFrame(processed_data.tolist(), index=df_optimized['PatienId']).reset_index()
la5 = pd.DataFrame(processed_data1.tolist(), index=df_optimized['PatienId']).reset_index()
la6a = pd.DataFrame(processed_dataa.tolist(), index=df_optimized['PatienId']).reset_index()
la6b = pd.DataFrame(processed_datab.tolist(), index=df_optimized['PatienId']).reset_index()
la6 = pd.concat([la6a,la6b],axis=0)